# Статанализ масок облаков и снега

В файл были экспортированы обучающие данные по облачности и снегу (см. https://176.9.38.120/cruncher/notebooks/deforestation/CMask_trainings.ipynb).

In [1]:
points = read.table('data/CMask_trainings/random_points.csv', header=TRUE, sep=';')

# summary(points)

Содержимое колонок:

  * cat -- идентификатор точки, в которой производились измерения;
  * l1_b1 -- l1_b11 -- данные LANDSAT8 (после атмосферной коррекции) по сцене LC81120282015365LGN00;
  * l2_b1 -- l2_b11 -- данные LANDSAT8 (после атмосферной коррекции) по сцене LC81130272015356LGN00;
  * t1, t2 -- данные по наземным объектам сцен LC81120282015365LGN00 и LC81130272015356LGN00 соответственно, значения: 1-облака;2-снег;3-остальное.
  * x, y -- координаты (UTM);

Соберем данные в dataframe для удобства анализа так, чтобы в нем одна строка соответствовала одному измерению, а не двум, как сейчас:

In [2]:
tmp1 = points[c('cat', 'x', 'y', 't1',
                'l1_b1', 'l1_b2', 'l1_b3', 'l1_b4', 'l1_b5', 'l1_b6', 
                'l1_b7', 'l1_b8', 'l1_b9', 'l1_b10', 'l1_b11'
               )]
names(tmp1) = c('cat', 'x', 'y', 't',
                'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11'
               )

tmp1$scene = 'S1' # 'LC81120282015365LGN00'

tmp2 = points[c('cat', 'x', 'y', 't2',
                'l2_b1', 'l2_b2', 'l2_b3', 'l2_b4', 'l2_b5', 'l2_b6', 
                'l2_b7', 'l2_b8', 'l2_b9', 'l2_b10', 'l2_b11'
               )]
names(tmp2) = c('cat', 'x', 'y', 't',
                'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 'b10', 'b11'
               )
tmp2$scene = 'S2' # 'LC81130272015356LGN00'


points = rbind(tmp1, tmp2)

points$t[points$t==0] = NA

points$scene = factor(points$scene)
points$t = factor(points$t)
levels(points$t) <- list(Cloud=1, Snow=2, Other=3)

points = na.omit(points)

summary(points)

      cat              x                y               t        
 Min.   :    1   Min.   :434113   Min.   :4979683   Cloud: 1579  
 1st Qu.:12508   1st Qu.:496221   1st Qu.:5060181   Snow : 3644  
 Median :24998   Median :536605   Median :5117104   Other:45316  
 Mean   :24998   Mean   :541089   Mean   :5112347                
 3rd Qu.:37496   3rd Qu.:586255   3rd Qu.:5165598                
 Max.   :50000   Max.   :664168   Max.   :5215814                
       b1               b2                b3                b4         
 Min.   :0.1124   Min.   :0.08453   Min.   :0.04529   Min.   :0.02953  
 1st Qu.:0.2011   1st Qu.:0.16832   1st Qu.:0.11290   1st Qu.:0.09587  
 Median :0.2291   Median :0.19891   Median :0.14421   Median :0.13350  
 Mean   :0.2529   Mean   :0.22642   Mean   :0.17388   Mean   :0.17051  
 3rd Qu.:0.2732   3rd Qu.:0.24862   3rd Qu.:0.19710   3rd Qu.:0.19912  
 Max.   :0.9407   Max.   :1.07539   Max.   :1.17382   Max.   :1.45950  
       b5               b6        

Видим, что в 9-м канале после атмосферной коррекции получились отрицательные значения. Объясняется это, по всей видимости, тем, что используемые в ходе коррекции константы (gain или bias, см. метаданные) содержали неподходящие для данной сцены значения.

In [3]:
# Число примеров, оставшихся после удаления отсутсвующих данных
dim(points)

[1] 50539    16

## Первый взгляд на данные

### Отражающая способность различных объектов

Построим ящики с усами по каждому каналу и сравним какие значения отражающей способности типичны для каждого типа поверхности.

In [4]:
opar = par(no.readonly = TRUE)      # make a copy of current settings

png("Img/CMASK/type_vs_bands.png", width=728, height=728, units="px")
    par(mfrow = c(3,4))
    boxplot(b1 ~ t, data=points, ylab='b1')
    boxplot(b2 ~ t, data=points, ylab='b2')
    boxplot(b3 ~ t, data=points, ylab='b3')
    boxplot(b4 ~ t, data=points, ylab='b4')

    boxplot(b5 ~ t, data=points, ylab='b5')
    boxplot(b6 ~ t, data=points, ylab='b6')
    boxplot(b7 ~ t, data=points, ylab='b7')
    boxplot(b8 ~ t, data=points, ylab='b8')

    boxplot(b9 ~ t, data=points, ylab='b9')
    boxplot(b10 ~ t, data=points, ylab='b10')
    boxplot(b11 ~ t, data=points, ylab='b11')
    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/type_vs_bands.png">

* 1-й--5-й и 8-й каналы ведут себя примерно одинаково: яркости пикслеей, соотвествующие снегу, выше, чем яркости облачных пикселей. А яркость пикселей облаков и снега обычно выше, чем пикселей других объектов. Однако, нельзя провести четкой границы -- области яркостей пикселей различных объектов довольно сильно пересекаются. Видно, что в этих каналах распределение имеет длинный правый хвост: на графиках видны выбросы в верхней части ящиков. Возможно, что стоит преобразовать каналы, чтобы получить более симметричное распределение. Поскольку восьмой канал - панхроматический, то он несет в основном пространственную информацию, а ничего нового с точки зрения спектральной информации не привносит. Поэтому в дальнейшем он буде опущен при анализе.
* 6-й--7-й каналы: высокое отражение в облачных областях, чуть меньшее в заснеженных и еще меньше для остальных объектов. Точно также данные показывают длинный правый хвост в распределении.
* 9-й канал: очень длинный правый хвост в облачных пикселях. Значения облачных пикселей в целом значительно выше, чем у остальных. Поскольку этот канал был разработан для исследования атмосферных эффектов, то, возможно, этот канал будет одним из основных маркеров облако/не облако.
* 10-й и 11-й каналы (температура) cхожи между собой: на них заметно, что температура облаков обычно выше, чем остальных объектов (зима?), хотя у облаков бросается в глаза очень длинный левый хвост распределения.

Преобразуем данные, чтобы получить более равномерные распределения. При этом выкинем из анализа 8-й канал, основную часть каналов прологарифмируем для избавления от длинных хвостов, 9-й канал возведем в степень, а 10-й и 11-й каналы оставим без изменений (длинный хвост только у облаков, а не у всего канала):

In [5]:
png("Img/CMASK/type_vs_logbands.png", width=728, height=728, units="px")
    par(mfrow = c(2, 4))
    boxplot(log(b1) ~ t, data=points, ylab='log(b1')
    boxplot(log(b2) ~ t, data=points, ylab='log(b2')
    boxplot(log(b3) ~ t, data=points, ylab='log(b3')
    boxplot(log(b4) ~ t, data=points, ylab='log(b4')

    boxplot(log(b5) ~ t, data=points, ylab='log(b5)')
    boxplot(log(b6) ~ t, data=points, ylab='log(b6)')
    boxplot(log(b7) ~ t, data=points, ylab='log(b7)')

    boxplot(I(b9^0.5) ~ t, data=points, ylab='sqrt(b9)')
    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/type_vs_logbands.png">

Видно, что распределения частично выровнялись, хотя местами по-прежнему остались длинные хвосты.

### Зависимость от сцены

Построим те же самые ящики с усами, но попробуем понять, насколько отличаются данные не только из-за объектов, но и из-за того, что они представлены в разных сценах.

In [6]:
png("Img/CMASK/scene_vs_logbands.png", width=728, height=728, units="px")
    par(mfrow = c(3, 4))
    boxplot(log(b1) ~ scene, data=points, ylab='log(b1)')
    boxplot(log(b2) ~ scene, data=points, ylab='log(b2)')
    boxplot(log(b3) ~ scene, data=points, ylab='log(b3)')
    boxplot(log(b4) ~ scene, data=points, ylab='log(b4)')

    boxplot(log(b5) ~ scene, data=points, ylab='log(b5)')
    boxplot(log(b6) ~ scene, data=points, ylab='log(b6)')
    boxplot(log(b7) ~ scene, data=points, ylab='log(b7)')

    boxplot(I(b9^0.5) ~ scene, data=points, ylab='sqrt(b9)')

    boxplot(b10 ~ scene, data=points, ylab='b10')
    boxplot(b11 ~ scene, data=points, ylab='b11')

    par(opar)
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_vs_logbands.png">

Итак, мы видим, что сцена 2 (LC81130272015356LGN00) более яркая, чем сцена 1 (LC81120282015365LGN00) во всех каналах. Вопрос в том, насколько велика эта вариативность данных, которая вносится разными условиями съемки, и не может ли оказаться так, что условия съемки влияют на яркость пикселей гораздо больше, чем тип обеъкта (см., например, различия якркостей в 10-м и 11-м каналах).

Для того, чтобы ответить на этот вопрос построим ящики для отдельных каналов.

#### Канал B1

В первом канале видно, что яркость снега может меняться в достаточно широких пределах в зависимости от сцены, настолько, что она яркости облаков и снежных участков могут пересекаться в большой степени.

In [7]:
png("Img/CMASK/scene_type_vs_b1.png", width=728, height=728, units="px")
    boxplot(log(b1) ~ scene+t, data=points, ylab='log(b1)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b1.png">

#### Канал B2

Аналогичная картина и во втором канале.

In [8]:
png("Img/CMASK/scene_type_vs_b2.png", width=728, height=728, units="px")
    boxplot(log(b2) ~ scene+t, data=points, ylab='log(b2)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b2.png">

#### Канал B3

Аналогично и в третьем:

In [9]:
png("Img/CMASK/scene_type_vs_b3.png", width=728, height=728, units="px")
    boxplot(log(b3) ~ scene+t, data=points, ylab='log(b3)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b3.png">

#### Канал B4

In [10]:
png("Img/CMASK/scene_type_vs_b4.png", width=728, height=728, units="px")
    boxplot(log(b4) ~ scene+t, data=points, ylab='log(b4)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b4.png">

#### Канал B5

In [11]:
png("Img/CMASK/scene_type_vs_b5.png", width=728, height=728, units="px")
    boxplot(log(b5) ~ scene+t, data=points, ylab='log(b5)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b5.png">

#### Канал B6

Зато 6-й канал, похоже, меньше зависит от сцены, чем предыдущие:

In [12]:
png("Img/CMASK/scene_type_vs_b6.png", width=728, height=728, units="px")
    boxplot(log(b6) ~ scene+t, data=points, ylab='log(b6)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b6.png">

#### Канал B7

Очень похож на 6-й канал, но вариативность данных внутри классов в нем чуть меньше.

In [13]:
png("Img/CMASK/scene_type_vs_b7.png", width=728, height=728, units="px")
    boxplot(log(b7) ~ scene+t, data=points, ylab='log(b7)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b7.png">

#### Канал B9

В 9-м канале вариативность яркостей облачных пикселей между сценами очень велика. Похоже, что от сцены тут зависит больше, чем от типа объекта.

In [14]:
png("Img/CMASK/scene_type_vs_b9.png", width=728, height=728, units="px")
    boxplot(I(b9^0.5) ~ scene+t, data=points, ylab='sqrt(b9)')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b9.png">

#### Канал B10

Аналогичная проблема облачных пикселей и в 10-м канале: от сцены зависит больше, чем от типа объекта, особенно ярко это проявляется для облаков.

In [15]:
png("Img/CMASK/scene_type_vs_b10.png", width=728, height=728, units="px")
    boxplot(b10 ~ scene+t, data=points, ylab='b10')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b10.png">

#### Канал B11

Та же самая проблема: от сцены зависит больше, чем от типа объекта.

In [16]:
png("Img/CMASK/scene_type_vs_b11.png", width=728, height=728, units="px")
    boxplot(b11 ~ scene+t, data=points, ylab='b11')
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/scene_type_vs_b11.png">

### Вывод

 * Для различения облаков и снега, видимо, необходимо использовать каналы 1--7. В каналах 9--11 также содержится полезная информация, но воспользоваться ей будет сложнее из-за большей вариативности из-за условий съемки.
 * Каналы 1--5 и каналы 6--7 образуют две группы переменных. Переменные в каждой группе ведут себя схожим образом (поведение каналов 1--5 очень похоже между собой для выбранных классов; аналогично с каналами 6 и 7). Поскольку многие методы очень чувствительны к корреляции объясняющих переменных, то есть смысл подумать о том, чтобы выбрать по одному каналу из каждой группы. Возможно такое уменьшение числа переменных (но и корреляции между ними) даст положительный эффект на точность распознавания.

## Простая модель

В этом разделе построим простейшую модель на основе логистической регрессии и посмотрим, насколько она решает задачу разделения интересующих нас объектов. Пойдем по следующей схеме работы:

1. Предварительный выбор объясняющих переменных. Объясняющих переменных (каналов, в нашем случае) много, но они сильно коррелируют между собой. Для большей устойчивости решения выберем несколько переменных, остальные пока проигнорируем (на следующем этапе оценим правильность выбора переменных).
2. Создание фиктивных переменных. Поскольку у нас выделено три объекта (класс "все остальное" назовем условно одним объектом), и они не упорядочены, то нам понадобится создать фиктивные переменные, которые мы будем использовать.
3. Построение моделей, отражающих связь между объясняющими переменными и зависимыми.
4. Исследуем модели, проверив, выполняются ли условия, на которые опирается процедура построения логистической регрессии.
5. Посмотрим, насколько изменится качество модели, если выбрать другой набор объясняющих переменных.

### Предварительный выбор объясняющих переменных

Посмотрим еще раз на взаимную корреляциию объясняющих переменных (для ускорения отрисовки сделаем небольшую подвыборку данных). 

In [17]:
psample_ind = sample(1:length(points[, 1]), size=500)
psample = points[psample_ind, ]

png("Img/CMASK/all_bands.png", width=728, height=728, units="px")
  pairs(~log(b1)+log(b2)+log(b3)+log(b4)+log(b5)+log(b6)+log(b7)+sqrt(b9)+b10+b11, data=psample, pch=19, cex=0.1) 
dev.off()

png 
  2

<img src="https://176.9.38.120/cruncher/files/deforestation/Img/CMASK/all_bands.png">

Итак, мы видим, что очень сильна корреляция между 6-м и 7-м каналами, затем между 10-м и 11-м каналами, а также между 1-м, 2-м, 3-м и 4-м каналами, на эти 4 канала довольно похож 5-й, но все-таки, видимо, его следует выделить в отдельную группу. Ту же самую картину мы увидим, если расчитаем коэффициент корреляции между этими переменными:

In [18]:
cor(data.frame(log(points$b1), log(points$b2), log(points$b3), log(points$b4), log(points$b5),
    log(points$b6), log(points$b7), sqrt(points$b9+0.001), points$b10, points$b11))

,log.points.b1.,log.points.b2.,log.points.b3.,log.points.b4.,log.points.b5.,log.points.b6.,log.points.b7.,sqrt.points.b9...0.001.,points.b10,points.b11
log.points.b1.,1.00000000,0.99643789,0.97640161,0.95345085,0.77579622,0.56200970,0.62302423,0.29965771,0.01198254,0.02497259
log.points.b2.,0.99643789,1.00000000,0.99004006,0.97379873,0.81480322,0.60138721,0.66103290,0.30740683,0.01602131,0.02772187
log.points.b3.,0.97640161,0.99004006,1.00000000,0.99516799,0.87531299,0.66887306,0.72502000,0.32957672,0.03250685,0.03846292
log.points.b4.,0.95345085,0.97379873,0.99516799,1.00000000,0.90528235,0.71371878,0.76711295,0.32894256,0.04238753,0.04732758
log.points.b5.,0.7757962,0.8148032,0.8753130,0.9052823,1.0000000,0.8443377,0.8717325,0.3428986,0.1084754,0.1026983
log.points.b6.,0.56200970,0.60138721,0.66887306,0.71371878,0.84433771,1.00000000,0.99188780,0.44287939,0.08266380,0.06304586
log.points.b7.,0.62302423,0.66103290,0.72502000,0.76711295,0.87173245,0.99188780,1.00000000,0.44975283,0.05640044,0.03886842
sqrt.points.b9...0.001.,0.2996577,0.3074068,0.3295767,0.3289426,0.3428986,0.4428794,0.4497528,1.0000000,-0.3242166,-0.4123782
points.b10,0.01198254,0.01602131,0.03250685,0.04238753,0.10847537,0.08266380,0.05640044,-0.32421662,1.00000000,0.98045369
points.b11,0.02497259,0.02772187,0.03846292,0.04732758,0.10269832,0.06304586,0.03886842,-0.41237817,0.98045369,1.00000000


### Фиктивные переменные

Возможно, есть хороший способ создания фиткивных переменных вместо одной зависимой переменной, но я на него не наткнулся, поэтому всю процедуру сделаю руками: введу три новых переменных answ1, answ2, answ3 которые будут кодировать тот факт, что пиксель принадлежит облачному, снежному или иному участку снимка соответственно. Для этого создам новый набор данных, в котором будут содержаться только интересующие нас переменные.

In [19]:
pdata = data.frame(answ1=(points$t=='Cloud'), answ2=(points$t=='Snow'), answ3=(points$t=='Other'), 
                   b1=points$b1, b5=points$b5, b7=points$b7)

### Построение моделей

Построим три модели (по числу фиктивных переменных) на базе логистической регрессии.

In [20]:
simple.glm1 = glm(answ1~log(b1)+log(b5)+log(b7), data=pdata, family='binomial')
simple.glm2 = glm(answ2~log(b1)+log(b5)+log(b7), data=pdata, family='binomial')
simple.glm3 = glm(answ3~log(b1)+log(b5)+log(b7), data=pdata, family='binomial')

Warning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurredWarning message:
: glm.fit: fitted probabilities numerically 0 or 1 occurred

Выпали предупрежедения о том, что вероятности очень близки к 1 или 0. Это не очень хороший знак. Будем разбираться дальше.

In [21]:
summary(simple.glm1)


Call:
glm(formula = answ1 ~ log(b1) + log(b5) + log(b7), family = "binomial", 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.5716  -0.0112  -0.0024  -0.0003   4.5273  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  22.9042     0.7465  30.683  < 2e-16 ***
log(b1)       4.0425     0.6865   5.888  3.9e-09 ***
log(b5)      -9.3558     0.5574 -16.784  < 2e-16 ***
log(b7)      14.7126     0.4827  30.477  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 14053.6  on 50538  degrees of freedom
Residual deviance:  1461.2  on 50535  degrees of freedom
AIC: 1469.2

Number of Fisher Scoring iterations: 11


In [22]:
summary(simple.glm2)


Call:
glm(formula = answ2 ~ log(b1) + log(b5) + log(b7), family = "binomial", 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.8715  -0.0743  -0.0243  -0.0073   3.3855  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)   3.0938     0.1986   15.58   <2e-16 ***
log(b1)       6.6087     0.2064   32.01   <2e-16 ***
log(b5)       7.9575     0.1998   39.82   <2e-16 ***
log(b7)      -3.8843     0.1096  -35.45   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 26183.7  on 50538  degrees of freedom
Residual deviance:  6011.9  on 50535  degrees of freedom
AIC: 6019.9

Number of Fisher Scoring iterations: 9


In [23]:
summary(simple.glm3)


Call:
glm(formula = answ3 ~ log(b1) + log(b5) + log(b7), family = "binomial", 
    data = pdata)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.5720   0.0006   0.0059   0.0364   4.3836  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -24.9639     0.4746 -52.602   <2e-16 ***
log(b1)     -13.3866     0.3108 -43.071   <2e-16 ***
log(b5)      -1.5586     0.1703  -9.154   <2e-16 ***
log(b7)      -4.1086     0.1225 -33.546   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 33595.6  on 50538  degrees of freedom
Residual deviance:  5664.6  on 50535  degrees of freedom
AIC: 5672.6

Number of Fisher Scoring iterations: 9


In [24]:
summary(simple.glm1$fitted.values)

     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
0.0000000 0.0000001 0.0000043 0.0312400 0.0000968 1.0000000 